In [5]:
import xarray as xr
import numpy as np
import sys, os, glob
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

os.chdir(root_dir)

from data_processing.libraries import mean_statistics, std_statistics, wind_power_density, weibull , weibull_statistics, wind_power_density, turbine_power, solar_power

In [9]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
region = 'Ireland'
run_dir = f'{root_dir}/CERRA/{region}/variablewise_files'

chunks={"time": 3600,"x": -1,"y": -1}
Epv = 0.216

In [11]:

ws = xr.open_dataset(f'{run_dir}/ws_10.nc',chunks=chunks)['ws']
t2m = xr.open_dataset(f'{run_dir}/t2m.nc',chunks=chunks)['t2m']
swdown = xr.open_dataset(f'{run_dir}/swdown.nc',chunks=chunks)['ssrd']

In [12]:
solar_power(ws, swdown,t2m, Epv)

<xarray.DataArray 'PVO' (time: 90337, y: 41, x: 46)>
dask.array<mul, shape=(90337, 41, 46), dtype=float32, chunksize=(3600, 41, 46), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1990-01-01 1990-01-01T03:00:00 ... 2021-01-01
    surface  float64 0.0
Dimensions without coordinates: y, x

In [17]:
ws

<xarray.DataArray 'ws' (time: 90337, y: 41, x: 46)>
dask.array<open_dataset-aec2b8498d7135d553cbb8e2ffdcf5ecws, shape=(90337, 41, 46), dtype=float32, chunksize=(3600, 41, 46), chunktype=numpy.ndarray>
Coordinates:
  * time               (time) datetime64[ns] 1990-01-01 ... 2021-01-01
    step               timedelta64[ns] ...
    heightAboveGround  float64 ...
    latitude           (y, x) float64 dask.array<chunksize=(41, 46), meta=np.ndarray>
    longitude          (y, x) float64 dask.array<chunksize=(41, 46), meta=np.ndarray>
    valid_time         (time) datetime64[ns] dask.array<chunksize=(3600,), meta=np.ndarray>
Dimensions without coordinates: y, x